<a href="https://colab.research.google.com/github/absolu674/Classic-Random-Goal/blob/main/AirtelPayinPayoutData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import os
import csv
import re

# Libraries to help with data visualization
import matplotlib.pyplot as plt
import seaborn as sns
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Usefull Functions

In [2]:
# Function to verify if a list is in the source data frame
# `source_df` is dataset
# `list` is the list to verify if they exist in `source_df`
# `source_df_col` is the column in the source_df to compare with
# `list_col` is the column in the list to compare with
def get_data_in_dataset(source_df, list, source_def_col='Reference', list_col='Reference'):
  # transform list into dataframe
  list_df = pd.DataFrame(list, columns=[list_col])
  merge_df = pd.merge(
    list_df,
    source_df,
    left_on=list_col,
    right_on=source_def_col,
    how='inner'  # 'inner' keep all rows that are in both datasets
  )
  return merge_df

In [3]:
# Function to read Excel or CSV files from folder and sub-folders and create a unique dataset
def files_to_dataset(dir_path, file_ext='xlsx', skip=0, new_header=0, separator=','):
    # Create an empty list to store datasets
    datasets = []

    # Traverse the directory and its subdirectories
    for root, _, files in os.walk(dir_path):
        # Filter files based on the file extension
        filtered_files = [file for file in files if file.endswith(file_ext)]

        # Read each file
        for file in filtered_files:
            file_path = os.path.join(root, file)

            try:
                # Read dataset
                if file_ext == 'xlsx':
                    df = pd.read_excel(file_path, skiprows=skip, header=new_header)
                elif file_ext == 'csv':
                    df = pd.read_csv(file_path, skiprows=skip, header=new_header, delimiter=separator)
                else:
                    print(f"Unsupported file extension: {file_ext}")
                    continue

                # Skip the last row
                df = df.iloc[:-1]

                # Print dataset shape
                print(f'File name: {file}, Path: {file_path}, Dataset shape: {df.shape}')

                datasets.append(df)

            except (pd.errors.ParserError, pd.errors.EmptyDataError, FileNotFoundError) as e:
                print(f"Error reading file {file} at {file_path}: {e}")

    # Merge all datasets into a single dataset
    if datasets:
        merged_dataset = pd.concat(datasets, ignore_index=True)
        # Print merged dataset shape
        print(f'Merged dataset shape: {merged_dataset.shape}')
        return merged_dataset
    else:
        print("No datasets found or successfully read.")
        return None

# Example usage:
# dataset = files_to_dataset('/path/to/directory', 'csv')

In [4]:
# Function to add date related columns: date, year, month, day_of_week, day, hour

def add_date_columns(df, column):
  # Convert date column to datetime
  df["date"] = pd.to_datetime(df[column], format='mixed')
  # Add year column
  df['year'] = df['date'].apply(lambda x: x.year)
  # Add month column
  df['month'] = df['date'].apply(lambda x: x.month)
  # Add day of the week column
  df['day_week'] = df['date'].apply(lambda x: x.dayofweek)
  # Add day column
  df['day'] = df['date'].apply(lambda x: x.day)
  # Add hour column
  df['hour'] = df['date'].apply(lambda x: x.hour)
  # Drop original date column
  df.drop(column, axis=1, inplace=True, errors='ignore')

  print(f'{df.info()}')

  return df

In [5]:
# Function to remove duplicated data based on unique column
def remove_duplicates_data(df, column):
  print(f'Shape before: {df.shape}')
  df_no_duplicates = df.drop_duplicates(subset=column)
  print(f'Shape after: {df_no_duplicates.shape}')
  return df_no_duplicates

In [6]:
# Function to count duplicates and display samples to illustrate
def count_duplicates(df, column):
  df_sorted = df.sort_values(by=column)
  duplicate_counts = df.duplicated(subset=[column]).sum()
  duplicates = df[df.duplicated(subset=[column], keep=False)]
  print(f'Number of duplicates: {duplicate_counts}')

In [7]:
# Function to display data period
def display_data_period(df):
  min = df['date'].min()
  max = df['date'].max()
  print(f'Start date: {min}, End date: {max}')

In [8]:
# Function to check if there are missing per period of 1 hour
def check_missing_data_on_period(df, starttime, endtime):
  min = df['date'].min()
  max = df['date'].max()

  # Generate an hourly datetime range between the min and max datetimes
  datetime_range = pd.date_range(start=max, end=min, freq='H')

  # Identify missing datetimes
  missing_datetimes = datetime_range[~datetime_range.isin(df['date'])]
  # Display the missing datetimes, if any
  if not missing_datetimes.empty:
    print("Missing datetimes:")
    print(missing_datetimes)
  else:
    print("No missing datetimes.")

In [9]:
# Plot number of unique customers per month
def unique_customer_plot(year, data, merchant='all', description='All but maxibet'):
  unique_customers_count = []
  months = np.arange(1, 13)

  for m in months:
    data_temp = data[data['merchant']==merchant] if merchant!='all' else data
    unique_count = data_temp[(data_temp['year'] == year) & (data_temp['month'] == m)]['phone'].nunique()
    transaction_count = data_temp[(data_temp['year'] == year) & (data_temp['month'] == m)]['phone'].count()
    unique_customers_count.append(unique_count)
    print(f'{merchant} in {year}/{m} unique customers {unique_count} total transactions {transaction_count}')

  plt.plot(months, unique_customers_count, "bx-")
  plt.xlabel("Month")  # Title of X-axis
  plt.ylabel("Unique customers count")  # Title of y-axis
  plt.title(f'{description} unique customers in {year}', fontsize=20)

  plt.show()

In [10]:
# transform phone number
def transform_phone(phone):
    # Check if phone ends with '.0'
    if phone.endswith('.0'):
      phone = re.sub(r'\.0$', '', phone)

    # Check if phone starts with '241' followed by 8 digits starting with 6 or 7
    if phone.startswith('241') and len(phone) == 12 and phone[3:5] in ('06', '07') and phone[4:].isdigit():
        # Replace '241' with '0'
        return phone[3:]

    # Check if phone starts with '241' followed by 8 digits starting with 6 or 7
    if phone.startswith('241') and len(phone) == 11 and phone[3] in ('6', '7') and phone[4:].isdigit():
        # Replace '241' with '0'
        return '0' + phone[3:]

    # Check if phone starts with '241' followed by 8 digits starting with 02, 04, 05, 06, or 07
    elif phone.startswith('241') and len(phone) == 11 and phone[0:5] in ('24102', '24104', '24105', '24106', '24107') and phone[5:].isdigit():
        # Replace specific patterns
        replacements = {'24102': '062', '24104': '074', '24105': '065', '24106': '066', '24107': '077'}
        return replacements.get(phone[0:5], phone) + phone[5:]

    # Check if phone starts with '241' followed by 7 digits starting with 2, 4, 5, 6, or 7
    elif phone.startswith('241') and len(phone) == 10 and phone[3] in ('2', '4', '5', '6', '7') and phone[4:].isdigit():
        # Replace specific patterns
        replacements = {'2412': '062', '2414': '074', '2415': '065', '2416': '066', '2417': '077'}
        return replacements.get(phone[0:4], phone) + phone[4:]

    # Check if phone has 8 digits starting with 6 or 7
    elif (len(phone) == 8 and phone[0] in ('6', '7') and phone.isdigit()):
        # Add '0' at the beginning
        return '0' + phone

    elif (len(phone) == 8 and phone[0:2] in ('02', '04', '05', '06', '07') and phone.isdigit()):
        # Replace specific digits
        replacements = {'02': '062', '04': '074', '05': '065', '06': '066', '07': '077'}
        return replacements.get(phone[0:2], phone) + phone[2:]

    # Check if phone has 7 digits starting with 2, 4, 5, 6, or 7
    elif len(phone) == 7 and phone[0] in ('2', '4', '5', '6', '7') and phone.isdigit():
        # Replace specific digits
        replacements = {'2': '062', '4': '074', '5': '065', '6': '066', '7': '077'}
        return replacements.get(phone[0], phone) + phone[1:]

    # If none of the above conditions match, return the original phone number
    return phone


In [11]:
!ls "/content/drive/Shared drives/RAPPORT OPERATEUR/AIRTEL/2025"

JANVIER


## Import Airtel Data

In [12]:
# airtel path
airtel_data_path = "/content/drive/Shared drives/RAPPORT OPERATEUR/AIRTEL/2025"

In [13]:
# airtel df
airtel_df = files_to_dataset(airtel_data_path, 'csv', separator=';')

File name: 01-01 Janvier .csv, Path: /content/drive/Shared drives/RAPPORT OPERATEUR/AIRTEL/2025/JANVIER/01-01 Janvier .csv, Dataset shape: (11442, 33)
File name: 02-02 Janvier 01.csv, Path: /content/drive/Shared drives/RAPPORT OPERATEUR/AIRTEL/2025/JANVIER/02-02 Janvier 01.csv, Dataset shape: (5882, 33)
File name: 02-02 Janvier 02.csv, Path: /content/drive/Shared drives/RAPPORT OPERATEUR/AIRTEL/2025/JANVIER/02-02 Janvier 02.csv, Dataset shape: (9660, 33)
File name: 02-02 Janvier 03.csv, Path: /content/drive/Shared drives/RAPPORT OPERATEUR/AIRTEL/2025/JANVIER/02-02 Janvier 03.csv, Dataset shape: (12225, 33)
File name: 03-03 Janvier 01.csv, Path: /content/drive/Shared drives/RAPPORT OPERATEUR/AIRTEL/2025/JANVIER/03-03 Janvier 01.csv, Dataset shape: (3954, 33)
File name: 03-03 Janvier 02.csv, Path: /content/drive/Shared drives/RAPPORT OPERATEUR/AIRTEL/2025/JANVIER/03-03 Janvier 02.csv, Dataset shape: (9923, 33)
File name: 04-04 Janvier 01.csv, Path: /content/drive/Shared drives/RAPPORT OP

<ipython-input-3-91a5f8cdd82d>:20: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, skiprows=skip, header=new_header, delimiter=separator)


File name: 04-04 Janvier R.Final.csv, Path: /content/drive/Shared drives/RAPPORT OPERATEUR/AIRTEL/2025/JANVIER/04-04 Janvier R.Final.csv, Dataset shape: (18452, 33)
File name: 05-05 Janvier 01.csv, Path: /content/drive/Shared drives/RAPPORT OPERATEUR/AIRTEL/2025/JANVIER/05-05 Janvier 01.csv, Dataset shape: (6666, 33)
File name: 05-05 Janvier R.Final.csv, Path: /content/drive/Shared drives/RAPPORT OPERATEUR/AIRTEL/2025/JANVIER/05-05 Janvier R.Final.csv, Dataset shape: (15884, 33)
File name: 06-06 Janvier 01.csv, Path: /content/drive/Shared drives/RAPPORT OPERATEUR/AIRTEL/2025/JANVIER/06-06 Janvier 01.csv, Dataset shape: (2043, 33)
File name: 06-06 Janvier R.Final.csv, Path: /content/drive/Shared drives/RAPPORT OPERATEUR/AIRTEL/2025/JANVIER/06-06 Janvier R.Final.csv, Dataset shape: (13537, 33)
File name: 07-07 Janvier R.Final.csv, Path: /content/drive/Shared drives/RAPPORT OPERATEUR/AIRTEL/2025/JANVIER/07-07 Janvier R.Final.csv, Dataset shape: (15345, 1)
File name: 03-03 Janvier R.Final.

<ipython-input-3-91a5f8cdd82d>:20: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, skiprows=skip, header=new_header, delimiter=separator)


File name: 11-11 Janvier R.Final.csv, Path: /content/drive/Shared drives/RAPPORT OPERATEUR/AIRTEL/2025/JANVIER/11-11 Janvier R.Final.csv, Dataset shape: (16530, 33)
File name: 12-12 Janvier (01).csv, Path: /content/drive/Shared drives/RAPPORT OPERATEUR/AIRTEL/2025/JANVIER/12-12 Janvier (01).csv, Dataset shape: (2675, 33)
File name: 12-12 Janvier 02.csv, Path: /content/drive/Shared drives/RAPPORT OPERATEUR/AIRTEL/2025/JANVIER/12-12 Janvier 02.csv, Dataset shape: (9025, 33)


<ipython-input-3-91a5f8cdd82d>:20: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path, skiprows=skip, header=new_header, delimiter=separator)


File name: 12-12 Janvier R.Final.csv, Path: /content/drive/Shared drives/RAPPORT OPERATEUR/AIRTEL/2025/JANVIER/12-12 Janvier R.Final.csv, Dataset shape: (17232, 33)
File name: 01-07 Janvier R.Final .csv, Path: /content/drive/Shared drives/RAPPORT OPERATEUR/AIRTEL/2025/JANVIER/Felicien/01-07 Janvier R.Final .csv, Dataset shape: (102432, 1)
Merged dataset shape: (415713, 35)


In [14]:
airtel_df.head()

,Record No,Transaction ID,Reference No.,Transaction Date & Time,Payer MFS Provider,Payer Payment Instrument,Payer Wallet Type/Linked Bank,Payer Bank Account No/Mobile No,Payer User Name,Sender Grade,...,Payer Previous Balance,Payer Post Balance,Payee Pre Balance,Payee Post Balance,Total Service Charge,External Transaction id,Receiver_name,Reason,",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,","Record No,Transaction ID,Reference No.,Transaction Date & Time,Payer MFS Provider,Payer Payment Instrument,Payer Wallet Type/Linked Bank,Payer Bank Account No/Mobile No,Payer User Name,Sender Grade,Payer Nick Name,Payer Mobile Number,Payer Category,Payee MFS Provider,Payee Payment Instrument,Payee Wallet Type/Linked Bank,Receiver Mobile Number,Payee Bank Account No/Mobile No,Receiver Category,Receiver Grade,Payee User Name,Payee Nick Name,Service Type,Transaction Status,Transaction Amount,Payer Previous Balance,Payer Post Balance,Payee Pre Balance,Payee Post Balance,Total Service Charge,External Transaction id,Receiver_name,Reason"
0,1,MP250101.0000.A65584,5565556327 maxibet241,01/01/2025 00:00,Airtel,WALLET,Normal,74568739.0,PIERRE MAGAMBOU,Gold Subscriber,...,NaN,NaN,1.273773e+09,1.273774e+09,8.0,COLL_BN682KANQE_288e4721a0fd01708185,NaN,NaN,NaN,NaN
1,2,MP250101.0000.A65612,5565556331 maxibet241,01/01/2025 00:00,Airtel,WALLET,Normal,74497648.0,YARA AMBROISE,Gold Subscriber,...,NaN,NaN,1.340817e+09,1.340819e+09,20.0,COLL_BN682KANQE_fe809e52ffd45190009d,NaN,NaN,NaN,NaN
2,3,CI250101.0001.A85321,disburs-BN682KANQE-2221301447,01/01/2025 00:01,Airtel,WALLET,Normal,77732629.0,DIGITECH AFRICA DIGITECH AFRICA,AGG ONLINEPORTAL,...,NaN,NaN,NaN,NaN,0.0,disburs-BN682KANQE-2221301447,NaN,NaN,NaN,NaN
3,4,MP250101.0001.B67372,5565556334 maxibet241,01/01/2025 00:01,Airtel,WALLET,Normal,74046754.0,MVE NKILI PIERRE ELVIS,Gold Subscriber,...,NaN,NaN,1.252719e+09,1.252719e+09,0.0,COLL_BN682KANQE_931887b831841e0804a8,NaN,NaN,NaN,NaN
4,5,MP250101.0001.A65663,5565556335 maxibet241,01/01/2025 00:01,Airtel,WALLET,Normal,77201875.0,SADIO KANTE,Gold Subscriber,...,NaN,NaN,1.266298e+09,1.266300e+09,20.0,COLL_BN682KANQE_9ec5801a3929b6c4148f,NaN,NaN,NaN,NaN


In [15]:
airtel_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 415713 entries, 0 to 415712
Data columns (total 35 columns):
 #   Column                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          Non-Null Count   Dtype  
---  ------                                                                                                                                                                                                                

In [16]:
# Columns from import
airtel_imported_columns = ['Transaction ID', 'Reference No.', 'Transaction Date & Time', 'Payer Mobile Number', 'Receiver Mobile Number', 'Transaction Amount', 'Transaction Status', 'Service Type']

In [17]:
airtel_df = airtel_df[airtel_imported_columns]

In [18]:
#adjust column name
airtel_df.rename(columns={'Reference No.': 'Reference Number', 'Transaction Status':'Status', 'Transaction Date & Time':'Transaction Date', 'Receiver Mobile Number': 'Payee Msisdn', 'Payer Mobile Number':'Payer Msisdn'}, inplace=True)

<ipython-input-18-1d72934f7115>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  airtel_df.rename(columns={'Reference No.': 'Reference Number', 'Transaction Status':'Status', 'Transaction Date & Time':'Transaction Date', 'Receiver Mobile Number': 'Payee Msisdn', 'Payer Mobile Number':'Payer Msisdn'}, inplace=True)


In [19]:
airtel_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 415713 entries, 0 to 415712
Data columns (total 8 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   Transaction ID      282648 non-null  object 
 1   Reference Number    282648 non-null  object 
 2   Transaction Date    282648 non-null  object 
 3   Payer Msisdn        282648 non-null  float64
 4   Payee Msisdn        282648 non-null  float64
 5   Transaction Amount  282648 non-null  float64
 6   Status              282648 non-null  object 
 7   Service Type        282648 non-null  object 
dtypes: float64(3), object(5)
memory usage: 25.4+ MB


In [20]:
airtel_df.head()

,Transaction ID,Reference Number,Transaction Date,Payer Msisdn,Payee Msisdn,Transaction Amount,Status,Service Type
0,MP250101.0000.A65584,5565556327 maxibet241,01/01/2025 00:00,74568739.0,77732629.0,800.0,Transaction Success,Merchant Payment
1,MP250101.0000.A65612,5565556331 maxibet241,01/01/2025 00:00,74497648.0,77732629.0,2000.0,Transaction Success,Merchant Payment
2,CI250101.0001.A85321,disburs-BN682KANQE-2221301447,01/01/2025 00:01,77732629.0,74739040.0,1500.0,Transaction Success,Cash in
3,MP250101.0001.B67372,5565556334 maxibet241,01/01/2025 00:01,74046754.0,77732629.0,0.0,Transaction Failed,Merchant Payment
4,MP250101.0001.A65663,5565556335 maxibet241,01/01/2025 00:01,77201875.0,77732629.0,2000.0,Transaction Success,Merchant Payment


In [21]:
# Adjust status value
airtel_df['Status'] = airtel_df['Status'].replace({'Transaction Success': 'TS', 'Transaction Failed': 'TF'})

In [22]:
airtel_df.head()

,Transaction ID,Reference Number,Transaction Date,Payer Msisdn,Payee Msisdn,Transaction Amount,Status,Service Type
0,MP250101.0000.A65584,5565556327 maxibet241,01/01/2025 00:00,74568739.0,77732629.0,800.0,TS,Merchant Payment
1,MP250101.0000.A65612,5565556331 maxibet241,01/01/2025 00:00,74497648.0,77732629.0,2000.0,TS,Merchant Payment
2,CI250101.0001.A85321,disburs-BN682KANQE-2221301447,01/01/2025 00:01,77732629.0,74739040.0,1500.0,TS,Cash in
3,MP250101.0001.B67372,5565556334 maxibet241,01/01/2025 00:01,74046754.0,77732629.0,0.0,TF,Merchant Payment
4,MP250101.0001.A65663,5565556335 maxibet241,01/01/2025 00:01,77201875.0,77732629.0,2000.0,TS,Merchant Payment


In [23]:
airtel_df.head()

,Transaction ID,Reference Number,Transaction Date,Payer Msisdn,Payee Msisdn,Transaction Amount,Status,Service Type
0,MP250101.0000.A65584,5565556327 maxibet241,01/01/2025 00:00,74568739.0,77732629.0,800.0,TS,Merchant Payment
1,MP250101.0000.A65612,5565556331 maxibet241,01/01/2025 00:00,74497648.0,77732629.0,2000.0,TS,Merchant Payment
2,CI250101.0001.A85321,disburs-BN682KANQE-2221301447,01/01/2025 00:01,77732629.0,74739040.0,1500.0,TS,Cash in
3,MP250101.0001.B67372,5565556334 maxibet241,01/01/2025 00:01,74046754.0,77732629.0,0.0,TF,Merchant Payment
4,MP250101.0001.A65663,5565556335 maxibet241,01/01/2025 00:01,77201875.0,77732629.0,2000.0,TS,Merchant Payment


In [24]:
# count duplicates
count_duplicates(airtel_df, 'Transaction ID')

Number of duplicates: 257482


In [25]:
# remove these duplicates
airtel_df = remove_duplicates_data(airtel_df, 'Transaction ID')

Shape before: (415713, 8)
Shape after: (158231, 8)


In [26]:
# format date
airtel_df=add_date_columns(airtel_df, 'Transaction Date')

<class 'pandas.core.frame.DataFrame'>
Index: 158231 entries, 0 to 313280
Data columns (total 13 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   Transaction ID      158230 non-null  object        
 1   Reference Number    158230 non-null  object        
 2   Payer Msisdn        158230 non-null  float64       
 3   Payee Msisdn        158230 non-null  float64       
 4   Transaction Amount  158230 non-null  float64       
 5   Status              158230 non-null  object        
 6   Service Type        158230 non-null  object        
 7   date                158230 non-null  datetime64[ns]
 8   year                158230 non-null  float64       
 9   month               158230 non-null  float64       
 10  day_week            158230 non-null  float64       
 11  day                 158230 non-null  float64       
 12  hour                158230 non-null  float64       
dtypes: datetime64[ns](1), float64(8), 

In [27]:
# change column type and format phone
airtel_df['Payer Msisdn'] = airtel_df['Payer Msisdn'].astype(str)
airtel_df['Payee Msisdn'] = airtel_df['Payee Msisdn'].astype(str)
airtel_df['Payer Msisdn'] = airtel_df['Payer Msisdn'].apply(transform_phone)
airtel_df['Payee Msisdn'] = airtel_df['Payee Msisdn'].apply(transform_phone)

## Airtel Payout

In [28]:
# airtel payout
airtel_payout = airtel_df[airtel_df['Service Type']=='Cash in'].copy()

In [29]:
airtel_payout['Reference'] = airtel_payout['Reference Number'].str.split('-').str[-1]
airtel_payout.head()

,Transaction ID,Reference Number,Payer Msisdn,Payee Msisdn,Transaction Amount,Status,Service Type,date,year,month,day_week,day,hour,Reference
2,CI250101.0001.A85321,disburs-BN682KANQE-2221301447,077732629,074739040,1500.0,TS,Cash in,2025-01-01 00:01:00,2025.0,1.0,2.0,1.0,0.0,2221301447
11,CI250101.0003.C84767,disburs-BN682KANQE-2221301449,077732629,074739040,1000.0,TS,Cash in,2025-01-01 00:03:00,2025.0,1.0,2.0,1.0,0.0,2221301449
17,CI250101.0004.A85372,disburs-BN682KANQE-2221301451,077732629,074157388,12000.0,TS,Cash in,2025-01-01 00:04:00,2025.0,1.0,2.0,1.0,0.0,2221301451
19,CI250101.0004.A85382,disburs-BN682KANQE-2221301452,077732629,074739040,1000.0,TS,Cash in,2025-01-01 00:04:00,2025.0,1.0,2.0,1.0,0.0,2221301452
21,CI250101.0004.A85390,disburs-BN682KANQE-2221301453,077732629,074465156,20000.0,TS,Cash in,2025-01-01 00:04:00,2025.0,1.0,2.0,1.0,0.0,2221301453


In [30]:
# payout columns
payout_columns = ['Transaction ID', 'Reference', 'Payee Msisdn', 'Transaction Amount', 'Service Type', 'Status', 'date']

In [31]:
# apply updated columns
airtel_payout = airtel_payout[payout_columns]

In [32]:
# count duplicates
count_duplicates(airtel_payout, 'Reference')

Number of duplicates: 0


In [33]:
airtel_payout.head()

,Transaction ID,Reference,Payee Msisdn,Transaction Amount,Service Type,Status,date
2,CI250101.0001.A85321,2221301447,074739040,1500.0,Cash in,TS,2025-01-01 00:01:00
11,CI250101.0003.C84767,2221301449,074739040,1000.0,Cash in,TS,2025-01-01 00:03:00
17,CI250101.0004.A85372,2221301451,074157388,12000.0,Cash in,TS,2025-01-01 00:04:00
19,CI250101.0004.A85382,2221301452,074739040,1000.0,Cash in,TS,2025-01-01 00:04:00
21,CI250101.0004.A85390,2221301453,074465156,20000.0,Cash in,TS,2025-01-01 00:04:00


In [34]:
airtel_payout[airtel_payout['Reference']=='2221306136']

,Transaction ID,Reference,Payee Msisdn,Transaction Amount,Service Type,Status,date
16558,CI250102.1453.C65368,2221306136,074298798,9950.0,Cash in,TS,2025-02-01 14:53:00


*texte en italique*## Airtel Payin

In [35]:
# airtel payin
airtel_payin = airtel_df[airtel_df['Service Type']=='Merchant Payment'].copy()
airtel_payin.head()

,Transaction ID,Reference Number,Payer Msisdn,Payee Msisdn,Transaction Amount,Status,Service Type,date,year,month,day_week,day,hour
0,MP250101.0000.A65584,5565556327 maxibet241,074568739,077732629,800.0,TS,Merchant Payment,2025-01-01 00:00:00,2025.0,1.0,2.0,1.0,0.0
1,MP250101.0000.A65612,5565556331 maxibet241,074497648,077732629,2000.0,TS,Merchant Payment,2025-01-01 00:00:00,2025.0,1.0,2.0,1.0,0.0
3,MP250101.0001.B67372,5565556334 maxibet241,074046754,077732629,0.0,TF,Merchant Payment,2025-01-01 00:01:00,2025.0,1.0,2.0,1.0,0.0
4,MP250101.0001.A65663,5565556335 maxibet241,077201875,077732629,2000.0,TS,Merchant Payment,2025-01-01 00:01:00,2025.0,1.0,2.0,1.0,0.0
5,MP250101.0001.C65257,5565556332 betafrica,077315605,077732629,2500.0,TS,Merchant Payment,2025-01-01 00:01:00,2025.0,1.0,2.0,1.0,0.0


In [36]:
# Add reference column
airtel_payin['Reference'] = airtel_payin['Reference Number'].str.split(' ').str[0]
airtel_payin.head()

,Transaction ID,Reference Number,Payer Msisdn,Payee Msisdn,Transaction Amount,Status,Service Type,date,year,month,day_week,day,hour,Reference
0,MP250101.0000.A65584,5565556327 maxibet241,074568739,077732629,800.0,TS,Merchant Payment,2025-01-01 00:00:00,2025.0,1.0,2.0,1.0,0.0,5565556327
1,MP250101.0000.A65612,5565556331 maxibet241,074497648,077732629,2000.0,TS,Merchant Payment,2025-01-01 00:00:00,2025.0,1.0,2.0,1.0,0.0,5565556331
3,MP250101.0001.B67372,5565556334 maxibet241,074046754,077732629,0.0,TF,Merchant Payment,2025-01-01 00:01:00,2025.0,1.0,2.0,1.0,0.0,5565556334
4,MP250101.0001.A65663,5565556335 maxibet241,077201875,077732629,2000.0,TS,Merchant Payment,2025-01-01 00:01:00,2025.0,1.0,2.0,1.0,0.0,5565556335
5,MP250101.0001.C65257,5565556332 betafrica,077315605,077732629,2500.0,TS,Merchant Payment,2025-01-01 00:01:00,2025.0,1.0,2.0,1.0,0.0,5565556332


In [37]:
# payin columns
payin_columns = ['Transaction ID', 'Reference', 'Payer Msisdn', 'Transaction Amount', 'Service Type', 'Status', 'date']

In [38]:
airtel_payin = airtel_payin[payin_columns]
airtel_payin.head()

,Transaction ID,Reference,Payer Msisdn,Transaction Amount,Service Type,Status,date
0,MP250101.0000.A65584,5565556327,074568739,800.0,Merchant Payment,TS,2025-01-01 00:00:00
1,MP250101.0000.A65612,5565556331,074497648,2000.0,Merchant Payment,TS,2025-01-01 00:00:00
3,MP250101.0001.B67372,5565556334,074046754,0.0,Merchant Payment,TF,2025-01-01 00:01:00
4,MP250101.0001.A65663,5565556335,077201875,2000.0,Merchant Payment,TS,2025-01-01 00:01:00
5,MP250101.0001.C65257,5565556332,077315605,2500.0,Merchant Payment,TS,2025-01-01 00:01:00


## Verifications

### Payouts verification

In [45]:
# list to verify
payout_ids = ["407373185"]

# result and row count
result_payout_df = get_data_in_dataset(airtel_payout, payout_ids)
count = result_payout_df['Reference'].count()

# show rows
result_payout_df.head(count)

,Reference,Transaction ID,Payee Msisdn,Transaction Amount,Service Type,Status,date


In [40]:
# result and row count
result_payout_df = get_data_in_dataset(airtel_payout, payout_ids)
count = result_payout_df['Reference'].count()
count

1

In [41]:
# show rows
result_payout_df.head(count)

,Reference,Transaction ID,Payee Msisdn,Transaction Amount,Service Type,Status,date
0,2221339903,CI250110.0917.B07961,077324473,10248.0,Cash in,TS,2025-10-01 09:17:00


### Payins verification

In [42]:
ebilling_ids = ["5565661174"]

In [43]:
# result and row count
result_payin_df = get_data_in_dataset(airtel_payin, ebilling_ids)
count = result_payin_df['Reference'].count()
count

1

In [44]:
# show rows
result_payin_df.head(count)

,Reference,Transaction ID,Payer Msisdn,Transaction Amount,Service Type,Status,date
0,5565661174,MP250106.1608.C98089,077297501,33619.0,Merchant Payment,TS,2025-06-01 16:08:00
